# DPAO model for N2O production

In [1]:
# package import
import pandas as pd
import qsdsan as qs
from qsdsan.utils import load_data

Set up components

In [2]:
# water
H2O = qs.Component(ID='H2O', search_ID='H2O',
                    particle_size='Soluble', degradability='Undegradable',
                    organic=False)

# substrate 

S_NO3 = qs.Component('S_NO3', search_ID='Nitrate', measured_as='N', phase='l',
                    particle_size='Soluble', degradability='Undegradable',
                    organic=False)

S_NO2 = qs.Component('S_NO2', search_ID="PubChem=946", measured_as='N', phase='l',
                    particle_size='Soluble', degradability='Undegradable',
                    organic=False)

S_NO = qs.Component('S_NO', search_ID='Nitric oxide', measured_as='N', phase='l',
                    particle_size='Soluble', degradability='Undegradable',
                    organic=False)

S_N2O = qs.Component('S_N2O', search_ID='Nitrous oxide', measured_as='N', phase='g',
                    particle_size='Dissolved gas', degradability='Undegradable',
                    organic=False)

S_N2 = qs.Component('S_N2', search_ID='Nitrogen gas', measured_as='N', phase='g',
                    particle_size='Dissolved gas', degradability='Undegradable',
                    organic=False)

S_O2 = qs.Component('S_O2', search_ID='Oxygen', measured_as='', phase='g',
                    particle_size='Dissolved gas', degradability='Undegradable',
                    organic=False)

S_PO4 = qs.Component('S_PO4', search_ID='Orthophosphate', measured_as='P', phase='l',
                    particle_size='Soluble', degradability='Undegradable',
                    organic=False)



## C source
S_S = qs.Component('S_S', search_ID='Acetate', measured_as = 'COD', phase='l',
                     particle_size = 'Soluble', degradability = 'Readily',
                     organic = True)

# particulates
X_DPAO = qs.Component('X_DPAO', formula = 'C5H7O2N', measured_as = 'COD', phase='l',
                     particle_size = 'Particulate', degradability = 'Biological',
                     organic = True)

X_PHA = qs.Component('X_PHA', measured_as = 'COD', phase='l',
                     particle_size = 'Particulate', degradability = 'Slowly',
                     organic = True)

X_PP = qs.Component('X_PP', measured_as = 'P', phase='l',
                     particle_size = 'Particulate', degradability = 'Undegradable',
                     organic = False)

X_I = qs.Component('X_I', measured_as = 'COD', phase='l',
                     particle_size = 'Particulate', degradability = 'Undegradable',
                     organic = True)

# make into components obj
cmps = qs.Components((H2O, S_NO3, S_NO2, S_NO, S_N2O, S_N2, S_PO4, S_O2, S_S, X_DPAO, X_PHA, X_PP, X_I))

In [3]:
# fill in cmps with some additional data, filling in thermosteam blanks with properties of water

for i in cmps:
    if i is H2O:
        continue # "continue" means skip the rest of the codes and continue with the next one in the loop
    i.default()
    i.copy_models_from(H2O, names=('sigma', 'epsilon', 'kappa', 'V', 'Cn', 'mu'))

cmps.compile()

# set as default components for the rest of the file
qs.set_thermo(cmps) 

Set up process

In [4]:
qs.Processes.load_from_file(path = "_process_liu.tsv", 
                            conserved_for=("C", "P"),
                            parameters=("Y_PO4", "Y_PHA", "Y_DPAO_NOX", "Y_DPAO_O2", "i_P_BM", "i_P_XI", "f_I",
                                         "q_PHA", "K_S_DPAO", "K_PP_DPAO", "q_PP", "K_PO4_PP", "K_PHA",
                                         "K_MAX_DPAO", "K_iPP_DPAO", "K_DPAO_PO4", 
                                         "mu_DPAO1", "mu_DPAO2", "mu_DPAO3", "mu_DPAO4", "mu_DPAO5",
                                         "K_NO3", "K_NO2", "K_NO", "K_N2O",
                                         "b_DPAO", "b_PP", "b_PHA", "K_NOX", "K_O2"),
                            compile = True)

CompiledProcesses([anaerobic storage of X_PHA, anoxic storage of X_PP on NO3-, anoxic storage of X_PP on NO2-, anoxic storage of X_PP on NO, anoxic storage of X_PP on N2O, anoxic growth on NO3-, anoxic growth on NO2-, anoxic growth on NO, anoxic growth on N2O, anoxic endogenous respiration on NO3-, anoxic endogenous respiration on NO2-, anoxic endogenous respiration on NO, anoxic endogenous respiration on N2O, anoxic respiration of X_PP on NO3-, anoxic respiration of X_PP on NO2-, anoxic respiration of X_PP on NO, anoxic respiration of X_PP on N2O, anoxic respiration of X_PHA on NO3-, anoxic respiration of X_PHA on NO2-, anoxic respiration of X_PHA on NO, anoxic respiration of X_PHA on N2O, aerobic storage of X_PP on O2, aerobic growth on O2, aerobic endogenous respiration on O2, aerobic respiration of X_PP on O2, aerobic respiration of X_PHA on O2])


Testing

In [5]:
ins1 = qs.WasteStream(H2O=100)
ins2 = ins1.copy()
ins2.imol["X_DPAO"] = 0.0011

In [11]:
su = qs.sanunits
su

<module 'qsdsan.sanunits' from '/Users/mckenna/miniconda3/envs/qsdsan/lib/python3.11/site-packages/qsdsan/sanunits/__init__.py'>